In [1]:
from statsbombpy import sb
import pandas as pd

In [2]:
flat_events = sb.events(match_id=3795107, split=True, flatten_attrs=False)
passes = pd.DataFrame(flat_events['passes'])

/home/agale/.local/lib/python3.8/site-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)


In [3]:
passes['outcome'] = [p['outcome']['name'] if 'outcome' in p else 'Complete' for p in passes['pass']]
passes['recipient_id'] = [p['recipient']['id'] if 'recipient' in p else None for p in passes['pass']]
passes['recipient_name'] = [p['recipient']['name'] if 'recipient' in p else None for p in passes['pass']]
passes['location_end'] = [p['end_location'] if 'end_location' in p else None for p in passes['pass']]
passes['height'] = [p['height']['name'] if 'height' in p else None for p in passes['pass']]
passes['type'] = [p['type']['name'] if 'type' in p else None for p in passes['pass']]
passes['body_part'] = [p['body_part']['name'] if 'body_part' in p else None for p in passes['pass']]
passes['location_x'] = [p[0] for p in passes['location']]
passes['location_y'] = [p[1] for p in passes['location']]
passes['location_end_x'] = [p[0] for p in passes['location_end']]
passes['location_end_y'] = [p[1] for p in passes['location_end']]

In [4]:
[p for p in passes['pass']][0:5]

[{'recipient': {'id': 5642, 'name': 'Axel Witsel'},
  'length': 6.87968,
  'angle': -2.5737486,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [54.2, 36.3],
  'type': {'id': 65, 'name': 'Kick Off'},
  'body_part': {'id': 40, 'name': 'Right Foot'}},
 {'recipient': {'id': 3077, 'name': 'Jan Vertonghen'},
  'length': 23.700844,
  'angle': -2.3890185,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [34.1, 17.5],
  'body_part': {'id': 40, 'name': 'Right Foot'}},
 {'recipient': {'id': 5632, 'name': 'Thorgan Hazard'},
  'length': 15.401948,
  'angle': -1.0170146,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [41.1, 1.6],
  'body_part': {'id': 38, 'name': 'Left Foot'}},
 {'recipient': {'id': 3509, 'name': 'Thibaut Courtois'},
  'length': 42.694847,
  'angle': 2.407559,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [5.4, 30.9],
  'body_part': {'id': 40, 'name': 'Right Foot'}},
 {'recipient': {'id': 20005, 'name': 'Toby Alderwe

In [5]:
passes.iloc[0]

id                                 474c5607-72d6-4d59-889f-5724d4698f48
index                                                                 5
period                                                                1
timestamp                                                  00:00:01.215
minute                                                                0
second                                                                1
type                                                           Kick Off
possession                                                            2
possession_team                                                 Belgium
play_pattern                                              From Kick Off
team                                                            Belgium
player                                             Romelu Lukaku Menama
position                                                 Center Forward
location                                                   [60.0

In [6]:
passes.iloc[0]['pass']

{'recipient': {'id': 5642, 'name': 'Axel Witsel'},
 'length': 6.87968,
 'angle': -2.5737486,
 'height': {'id': 1, 'name': 'Ground Pass'},
 'end_location': [54.2, 36.3],
 'type': {'id': 65, 'name': 'Kick Off'},
 'body_part': {'id': 40, 'name': 'Right Foot'}}

In [7]:
passes = passes.drop(columns=['related_events', 'pass', 'location', 'location_end'])

In [8]:
passes.to_csv('passes.csv', index=False)